# exp001
最頻値などのナイーブな予測

In [1]:
import os
import sys
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing

import pandas as pd
import numpy as np
import itertools

In [2]:
EXP_NAME = "exp001_valid"

In [3]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

# os.makedirs(os.path.join(OUTPUT_DIR, EXP_NAME), exist_ok=True)

In [4]:
# データ読み込み
train_sessions = pd.read_pickle(PREP_DIR + "train_sessions_week1.pkl")
labels = pd.read_pickle(PREP_DIR + "labels_week1.pkl")

In [5]:
sessions = train_sessions["session"].unique()

In [6]:
# sessionごとの最頻値
session_aid_cnt = train_sessions.groupby(["session", "aid"]).agg(cnt=("ts", "count")).reset_index()
session_aid_cnt = session_aid_cnt.sort_values(["session", "cnt"], ascending=[True, False])
session_recom = session_aid_cnt.groupby("session").head(20)
session_recom.head()

,session,aid,cnt
24,0,1649869,4
1,0,305831,3
6,0,461689,3
14,0,1110548,2
18,0,1190046,2


In [7]:
# 全体の最頻値
total_aid_cnt = train_sessions.groupby("aid").agg(cnt=("ts", "count")).reset_index()
total_aid_cnt = total_aid_cnt.sort_values("cnt", ascending=False)
total_aid_cnt = total_aid_cnt.head(20)
most_freq_aid = total_aid_cnt["aid"].tolist()
total_recom = pd.DataFrame(list(itertools.product(sessions, most_freq_aid)), columns=["session", "aid"])
total_recom.head()

,session,aid
0,0,1460571
1,0,1502122
2,0,670006
3,0,29735
4,0,1083665


In [8]:
# sessionごとと全体の結合(sessionを優先)
total_recom["cnt"] = 0
recom = pd.concat([session_recom, total_recom])
recom = recom.sort_values(["session", "cnt"], ascending=[True, False])
recom = recom.drop_duplicates(subset=["session", "aid"])
recom = recom.groupby("session").head(20)
recom = recom.groupby("session")["aid"].apply(list).reset_index()

In [9]:
click_preds = recom["aid"].tolist()
carts_preds = recom["aid"].tolist()
orders_preds = recom["aid"].tolist()

In [10]:
click_labels = labels["click_labels"].tolist()
carts_labels = labels["carts_labels"].tolist()
orders_labels = labels["orders_labels"].tolist()

In [29]:
def evaluate(click_labels, carts_labels, orders_labels, 
             click_preds, carts_preds, orders_preds, k=20):

    num_clicks = 0
    num_carts = 0
    num_orders = 0
    hit_clicks = 0
    hit_carts = 0
    hit_orders = 0

    for i in range(len(labels)):
        click_label = click_labels[i]
        carts_label = carts_labels[i]
        orders_label = orders_labels[i]
        click_pred = click_preds[i][:k]
        carts_pred = carts_preds[i][:k]
        orders_pred = orders_preds[i][:k]

        if not np.isnan(click_label):
            num_clicks += 1
            hit_clicks += int(click_label in click_pred)

        if type(carts_label) == list:
            num_carts += min(len(carts_label), k)
            hit_carts += len(set(carts_pred) & set(carts_label))
            
        if type(orders_label) == list:
            num_orders += min(len(orders_label), k)
            hit_orders += len(set(orders_pred) & set(orders_label))


    recall_clicks = hit_clicks / num_clicks
    recall_carts = hit_carts / num_carts
    recall_orders = hit_orders / num_orders
    score = (recall_clicks * 0.10) + (recall_carts * 0.30) + (recall_orders * 0.60)

    results = {}
    results["num_clicks"] = num_clicks
    results["hit_clicks"] = hit_clicks
    results["recall_clicks"] = recall_clicks
    results["num_carts"] = num_carts
    results["hit_carts"] = hit_carts
    results["recall_carts"] = recall_carts
    results["num_orders"] = num_orders
    results["hit_orders"] = hit_orders
    results["recall_orders"] = recall_orders
    results["score"] = score

    return results

In [30]:
evaluate(click_labels=labels["click_labels"].tolist(),
         carts_labels=labels["carts_labels"].tolist(),
         orders_labels=labels["orders_labels"].tolist(),
         click_preds=recom["aid"].tolist(),
         carts_preds=recom["aid"].tolist(),
         orders_preds=recom["aid"].tolist())

{'num_clicks': 4320369,
 'hit_clicks': 1330923,
 'recall_clicks': 0.30805771451466296,
 'num_carts': 1831983,
 'hit_carts': 472470,
 'recall_carts': 0.2579008648006013,
 'num_orders': 885029,
 'hit_orders': 462069,
 'recall_orders': 0.5220947562170279,
 'score': 0.42143288462186346}

In [24]:
print("click : ", num_clicks, hit_clicks, recall_clicks)
print("cart : ", num_carts, hit_carts, recall_carts)
print("order : ", num_orders, hit_orders, recall_orders)
print(score)


click :  4320369 1330923 0.30805771451466296
cart :  1831983 472470 0.2579008648006013
order :  885029 462069 0.5220947562170279
0.42143288462186346
